In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime as dt
import math
from sympy import Symbol, solveset, Interval
import numpy as np

from quant_functions import anal_funcs, data_funcs

# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)

In [31]:
def cal_ret(k, s, c, t):
    '''
    * k: 목표 자산, s: 시드 머니, c: 현금 흐름, t: 투자 기간
    * r: 목표수익률
    '''
    r=Symbol('r')
    equation = s*((1+r)**(t)) + c*((1-((1+r)**(t)))/(1-(1+r))) - k
    val = solveset(equation, r, Interval(0, 9999))
    try:
        solve_r = float(list(val)[0])
        ret = (np.round(solve_r, 6))*100
    except:
        ret = np.nan
    return ret

##### Target Growth

In [38]:
k_ls = [10000, 30000, 50000, 100000, 200000]
s_ls = [0, 1000, 5000, 10000, 30000]
c_ls = [50*12, 100*12, 200*12, 400*12]
t_ls = [3, 5, 10, 15, 20]

results = []
idxs = []
for s in s_ls:
    for k in k_ls:
        for t in t_ls:
            for c in c_ls:
                idxs.append((k, s, c, t))
                print((k, s, c, t))
                result = cal_ret(k, s, c, t)
                results.append(result)

(10000, 0, 600, 3)
(10000, 0, 1200, 3)
(10000, 0, 2400, 3)
(10000, 0, 4800, 3)
(10000, 0, 600, 5)
(10000, 0, 1200, 5)
(10000, 0, 2400, 5)
(10000, 0, 4800, 5)
(10000, 0, 600, 10)
(10000, 0, 1200, 10)
(10000, 0, 2400, 10)
(10000, 0, 4800, 10)
(10000, 0, 600, 15)
(10000, 0, 1200, 15)
(10000, 0, 2400, 15)
(10000, 0, 4800, 15)
(10000, 0, 600, 20)
(10000, 0, 1200, 20)
(10000, 0, 2400, 20)
(10000, 0, 4800, 20)
(30000, 0, 600, 3)
(30000, 0, 1200, 3)
(30000, 0, 2400, 3)
(30000, 0, 4800, 3)
(30000, 0, 600, 5)
(30000, 0, 1200, 5)
(30000, 0, 2400, 5)
(30000, 0, 4800, 5)
(30000, 0, 600, 10)
(30000, 0, 1200, 10)
(30000, 0, 2400, 10)
(30000, 0, 4800, 10)
(30000, 0, 600, 15)
(30000, 0, 1200, 15)
(30000, 0, 2400, 15)
(30000, 0, 4800, 15)
(30000, 0, 600, 20)
(30000, 0, 1200, 20)
(30000, 0, 2400, 20)
(30000, 0, 4800, 20)
(50000, 0, 600, 3)
(50000, 0, 1200, 3)
(50000, 0, 2400, 3)
(50000, 0, 4800, 3)
(50000, 0, 600, 5)
(50000, 0, 1200, 5)
(50000, 0, 2400, 5)
(50000, 0, 4800, 5)
(50000, 0, 600, 10)
(50000, 

In [44]:
target_rets = pd.DataFrame(idxs, columns=['목표자산', '초기자산', '현금흐름', '투자기간']).astype('int64')
target_rets['현금흐름'] = target_rets['현금흐름']//12
target_rets[['목표자산', '초기자산', '현금흐름']] = (target_rets[['목표자산', '초기자산', '현금흐름']]).astype('int64')
target_rets['목표수익률'] = results
target_rets['목표수익률'] = target_rets['목표수익률'].fillna(0)*0.01
target_rets = target_rets[['초기자산', '현금흐름', '투자기간', '목표자산', '목표수익률']].sort_values(['초기자산', '현금흐름', '투자기간', '목표자산'])

In [45]:
target_rets.to_excel(f"target_growth.xlsx", index=False)